In [2]:
table_count = 1

# Table 1

Description of samples in study, %PDX1+/NKX6-1, and UUIDs for all sequencing data

In [4]:
t1 = read.csv("../Table/PPC_pilot_sequenced_samples.csv")

# Table 2

Demuxlet results for batch 1 cryopreserved scRNA-seq

In [ ]:
t2 = read.table("/projects/PPC/pipeline/scRNA-Seq/190708_K00180_0854_AH75KHBBXY/demuxlet/out.best", header = T)

# Table 3

Demuxlet results for batch 2 cryopreserved scRNA-seq

In [15]:
t3 = read.table("/projects/PPC/pipeline/scRNA-Seq/190708_K00180_0854_AH75KHBBXY/demuxlet/out.best", header = T)

In [17]:
# head(t3)